<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Vectorización


In [1]:
import numpy as np

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [3]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [10]:
terminos = []
for c in corpus:
  terminos.append(c.split())

terminos

[['que', 'dia', 'es', 'hoy'],
 ['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'],
 ['martes', 'muchas', 'gracias']]

In [15]:
terminos_set = [item for sublist in terminos for item in sublist]
terminos_set = set(terminos_set)
terminos_set

{'de', 'dia', 'el', 'es', 'gracias', 'hoy', 'martes', 'muchas', 'que'}

In [16]:
def get_terminos_set(corpus):
  terminos = []
  for c in corpus:
    terminos.append(c.split())

  terminos_set = [item for sublist in terminos for item in sublist]
  terminos_set = set(terminos_set)
  return terminos_set

### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [37]:
corpus[0].split()

['que', 'dia', 'es', 'hoy']

In [43]:
def get_oneHotEnc(corpus):
  terminos_set = get_terminos_set(corpus)

  one_hots = []
  for c in corpus:
    one_hot = []
    for t in terminos_set:
      if ' '+t+' ' in ' '+c+' ':
        one_hot.append(1)
      else:
        one_hot.append(0)
    one_hots.append(np.array(one_hot))

  return np.array(one_hots)

In [44]:
m = get_oneHotEnc(corpus)
m, terminos_set

(array([[0, 0, 1, 0, 0, 0, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 1, 1, 0],
        [1, 0, 0, 0, 1, 1, 0, 0, 0]]),
 {'de', 'dia', 'el', 'es', 'gracias', 'hoy', 'martes', 'muchas', 'que'})

In [45]:
import pandas as pd
pd.DataFrame(m,columns=list(terminos_set),index=corpus)

,martes,el,es,de,gracias,muchas,hoy,dia,que
que dia es hoy,0,0,1,0,0,0,1,1,1
martes el dia de hoy es martes,1,1,1,1,0,0,1,1,0
martes muchas gracias,1,0,0,0,1,1,0,0,0


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [77]:
def get_freq_vects(corpus):
  terminos_set = get_terminos_set(corpus)

  freqs = []
  for c in corpus:
    freq = []
    for t in terminos_set:
      freq.append(c.split().count(t))

    freqs.append(np.array(freq))

  return np.array(freqs), terminos_set

In [78]:
freqs, _ = get_freq_vects(corpus)
freqs

array([[0, 0, 1, 0, 0, 0, 1, 1, 1],
       [2, 1, 1, 1, 0, 0, 1, 1, 0],
       [1, 0, 0, 0, 1, 1, 0, 0, 0]])

In [60]:
import pandas as pd
pd.DataFrame(freqs,columns=list(terminos_set),index=corpus)

,martes,el,es,de,gracias,muchas,hoy,dia,que
que dia es hoy,0,0,1,0,0,0,1,1,1
martes el dia de hoy es martes,2,1,1,1,0,0,1,1,0
martes muchas gracias,1,0,0,0,1,1,0,0,0


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [75]:
def get_idf(corpus, term):
  N = corpus.shape[0]
  DF = 0

  for c in corpus:
    if ' '+term+' ' in ' '+c+' ':
      DF +=1

  return np.log10(N/DF)


In [84]:
get_idf(corpus,'gracias')

0.47712125471966244

In [99]:
def get_tf_idf(corpus):
  freqs, terminos_set = get_freq_vects(corpus)

  idfs = [get_idf(corpus,t) for t in terminos_set]

  tf_idf = freqs * idfs
  return tf_idf, terminos_set

In [103]:
tf_idf, terminos_set = get_tf_idf(corpus)
tf_idf

array([[0.        , 0.        , 0.17609126, 0.        , 0.        ,
        0.        , 0.17609126, 0.17609126, 0.47712125],
       [0.35218252, 0.47712125, 0.17609126, 0.47712125, 0.        ,
        0.        , 0.17609126, 0.17609126, 0.        ],
       [0.17609126, 0.        , 0.        , 0.        , 0.47712125,
        0.47712125, 0.        , 0.        , 0.        ]])

In [104]:
df = pd.DataFrame(tf_idf,columns=list(terminos_set),index=corpus)
df

,martes,el,es,de,gracias,muchas,hoy,dia,que
que dia es hoy,0.000000,0.000000,0.176091,0.000000,0.000000,0.000000,0.176091,0.176091,0.477121
martes el dia de hoy es martes,0.352183,0.477121,0.176091,0.477121,0.000000,0.000000,0.176091,0.176091,0.000000
martes muchas gracias,0.176091,0.000000,0.000000,0.000000,0.477121,0.477121,0.000000,0.000000,0.000000


In [111]:
df.loc['que dia es hoy'].values

array([0.        , 0.        , 0.17609126, 0.        , 0.        ,
       0.        , 0.17609126, 0.17609126, 0.47712125])

### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [135]:
def get_simil(corpus, idx):
  tf_idf, terminos_set = get_tf_idf(corpus)
  df = pd.DataFrame(tf_idf,columns=list(terminos_set),index=corpus)

  doc = corpus[idx]
  doc_vec = df.loc[doc].values
  corpus = corpus[corpus!=doc]

  simils = [[1,doc]]
  for c in corpus:
    c_vec = df.loc[c].values
    simil = cosine_similarity(doc_vec,c_vec)
    simils.append([simil,c])

  simils.sort(key=lambda x: x[0], reverse=True)
  return simils

In [138]:
get_simil(corpus, 2)

[[1, 'martes muchas gracias'],
 [0.10845711727883083, 'martes el dia de hoy es martes'],
 [0.0, 'que dia es hoy']]

In [142]:
corpus1 = np.array(['La tecnologia es una herramienta en si misma',
                   'Con una herramienta uno puede construir cosas',
                   'si sale el sol va a estar lindo',
                   'un día lindo es un buen dia',
                   'el perro es feliz en un sillón soleado'])

In [143]:
get_simil(corpus1,0)

[[1, 'La tecnologia es una herramienta en si misma'],
 [0.13007684606237055, 'Con una herramienta uno puede construir cosas'],
 [0.08995500631964329, 'el perro es feliz en un sillón soleado'],
 [0.06324894545860953, 'si sale el sol va a estar lindo'],
 [0.022109114260899943, 'un día lindo es un buen dia']]

In [145]:
get_simil(corpus1,3)

[[1, 'un día lindo es un buen dia'],
 [0.15304094991271058, 'el perro es feliz en un sillón soleado'],
 [0.06103928619174247, 'si sale el sol va a estar lindo'],
 [0.022109114260899943, 'La tecnologia es una herramienta en si misma'],
 [0.0, 'Con una herramienta uno puede construir cosas']]

**Conclusiones:**

Se evalúa el desempeño del mecanismo de TF-IDF para establecer la similaridad de documentos en un corpus.

Como principal ventaja se puede destacar la velocidad y la posibilidad de implementarlo con pocas librerías.

Y como principal desventaja está el hecho de que se basa en la ocurrencia de palabras en el documento. Por un lado, no concidera errores en las palabras y por otro no se captura ningún "fenómeno" gramatical, como sustantivos, adjetivos, intenión, etc.